In [35]:
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
# Load a pre-trained YOLOv8 model
model = YOLO('./models/best.pt')
video_path = './Assignment Materials/Assignment Materials/15sec_input_720p.mp4'
import torchreid

feature_model = torchreid.models.build_model(
    name='osnet_x0_25',
    num_classes=1000,
    pretrained=True
)
model.eval().to('cuda')


Successfully loaded imagenet pretrained weights from "C:\Users\Abimanyu/.cache\torch\checkpoints\osnet_x0_25_imagenet.pth"


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 80, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2), bias=False)
        (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3(
        (cv1): Conv(
          (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_runnin

In [36]:
!nvidia-smi

Wed Jun 25 01:10:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.14                 Driver Version: 566.14         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   57C    P8              1W /   50W |    1225MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [45]:
tracker = DeepSort(max_age=5, n_init=10, max_cosine_distance=0.8, half=True, bgr=True, embedder="torchreid", embedder_model_name="osnet_ain_x1_0")

import sys
# Check if the model is loaded correctly
if model is None:
    print("Error: Model not loaded.")
    sys.exit(1)

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

frame_count = 0

while True:
    frame_count += 1
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break
    
    if frame_count == 1 or frame_count % 3 != 0:
        results = model(frame)
        boxes = results[0].boxes if results else None

        detections = []

        print("Gathering detections...")
        if boxes is None or len(boxes) == 0:
            print("No detections found.")
            continue
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1
            cls = box.cls[0]
            if cls != 2:
                continue
            conf = box.conf[0]
            detections.append([(x1, y1, w, h), cls, conf])
        
        # Update Deep SORT tracker
    tracks = tracker.update_tracks(detections, frame=frame)

    # Draw tracks
    for i, track in enumerate(tracks):
        if not track.is_confirmed():
            continue
        ltrb = track.to_ltrb()
        print(f"ltrb: {ltrb}")
        track_id = track.track_id

        if len(previous_tracks) > 0:
            # Compare with all previous tracks to identify if the current track is a continuation of a previous one
            for j, prev_track in enumerate(previous_tracks):
                comp_x1, comp_y1, comp_x2, comp_y2 = prev_track.to_ltrb()
                # Calculate the center of the top edge of the previous track and the current track
                prev_center_x = (comp_x1 + comp_x2) / 2
                prev_center_y = comp_y1
                current_center_x = (ltrb[0] + ltrb[2]) / 2
                current_center_y = ltrb[1]
                # Assign previous track ID if the current track is close enough to a previous one
                if abs(prev_center_x - current_center_x) < 50 and abs(prev_center_y - current_center_y) < 50:
                    track_id =  prev_track.track_id
                    # Remove the previous track from the list to avoid reusing it
                    previous_tracks.pop(j)

        cv2.rectangle(frame, (int(ltrb[0]), int(ltrb[1])), (int(
            ltrb[2]), int(ltrb[3])), (0, 0, 255), 2)
        cv2.putText(frame, f"{str(track_id)}", (int(ltrb[0]), int(
            ltrb[1] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    previous_tracks = tracks.copy()

    cv2.imshow('YOLOv11 Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Exiting video stream.")
        break


cv2.destroyAllWindows()
cap.release()

print("Detections gathered, tracking...")

c:\Users\Abimanyu\miniconda3\envs\ptga\lib\site-packages\torchreid\utils\tools.py:43: UserWarning: No file found at "c:\Users\Abimanyu\miniconda3\envs\ptga\lib\site-packages\deep_sort_realtime\embedder\weights\osnet_ain_ms_d_c_wtsonly.pth"
  warnings.warn('No file found at "{}"'.format(fpath))


Successfully loaded imagenet pretrained weights from "C:\Users\Abimanyu/.cache\torch\checkpoints\osnet_ain_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: osnet_ain_x1_0
- params: 2,193,616
- flops: 978,878,352

0: 384x640 1 ball, 16 players, 2 referees, 281.2ms
Speed: 4.7ms preprocess, 281.2ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)
Gathering detections...

0: 384x640 18 players, 2 referees, 61.6ms
Speed: 3.0ms preprocess, 61.6ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)
Gathering detections...

0: 384x640 1 ball, 14 players, 2 referees, 64.1ms
Speed: 3.1ms preprocess, 64.1ms inference, 3.9ms postprocess per image at shape (1, 3, 384, 640)
Gathering detections...

0: 384x640 1 ball, 14 players, 2 referees, 62.5ms
Speed: 2.7ms preprocess, 62.5ms inference, 3.4ms postprocess per image at shape (1, 3, 384, 640)
Gathering detections...

0: 384x640 1